In [1]:

import scipy,os,math,random,json
import flask,django
from flask import Flask,request,render_template_string

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.widgets import Slider,Button,RadioButtons

from IPython.display import Image,display
from IPython.core.display import HTML 

import ipywidgets as widgets
from ipywidgets import IntSlider,interact,interactive,fixed,interact_manual
from ipywidgets.embed import embed_minimal_html,embed_data

from markupsafe import escape
from nbconvert.exporters import HTMLExporter
from openpyxl.workbook import Workbook

def cons(k,l):#array constante de término k y longitud l
    x=[]
    for i in range(l):
        x=x+[k]
    return(x)

def zero(l):#array constante de término 0 y longitud l
    x=[]
    for i in range(l):
        x=x+[0]
    return(x)

def extract(A):#extrae, del array de arrays A, un array con el primer elemento de cada array de A
    E=[]
    for i in range(len(A)):
        E=E+[A[i][0]]
    return E

def addJ(A,J):#añade J a las componentes del array A
    B=[]
    for j in range(len(A)):
        B=B+[A[j]+J]
    return(B)
    
def col(A,j):#extrae un array con el elemento j de cada array de un array A
    column=[]
    for i in range(len(A)):
        if j+1<=len(A[i]):
            column=column+[A[i][j]]
        else:
            column=column+[]
    return(column)

def switch(X,j,x):#cambia la componente j del array X por x 
    Y=[]
    for i in range(len(X)):
        if i==j:
            Y=Y+[x]
        else:
            Y=Y+[X[i]]
    return(Y)

def switchcol(A,j,C):#cambia la columna j del array de arrays A por la columna C
    B=[]
    for i in range(len(A)):
        B=B+[switch(A[i],j,C[i])]
    return(B)   

def superswitch(X,J,C):#va cambiando recursivamente la columna J[j] del array de arrays A por las componente C[j] del array C
    Y=[]
    for j in range(len(J)):
        X=switch(X,J[j],C[j])
    return(Y)

def lc(W):#devuelve la longitud del array más grande del array de arrays W
    l=[]
    for n in range(len(W)):
        l=l+[len(W[n])]
    L=max(l)
    return(L)

def trans(W):#transpone el array de arrays W
    l=lc(W)
    O=[]
    for h in range(0,l):
        O=O+[col(W,h)]
    return(O)
    
def tale(X,k):#corta la cola del array X a partir del índice k
    Y=[]
    for i in range(0,k+1):
        Y=Y+[X[i]]
    return(Y)

def inter(G,H):#pone dos arrays G y H como columnas adyacentes y vuelta a array
    K=[]
    for i in range(min(len(G),len(H))):
        K=K+[[G[i],H[i]]]
    return(K)

def scaprod(X,Y):#producto escalar de dos arrays X e Y
    l=min(len(X),len(Y))
    s=0
    for i in range(l):
        s=s+X[i]*Y[i]
    return(s)

def triscaprod(X,Y,Z):#producto escalar de tres arrays X, Y y Z
    l=min(len(X),len(Y),len(Z))
    s=0
    for i in range(l):
        s=s+X[i]*Y[i]*Z[i]
    return(s)

def joint(U,V):#suma dos arrays U y V componente a componente
    T=[]
    le=min(len(U),len(V))
    for l in range(le):
        T=T+[U[l]+V[l]]
    return(T)

def algprod(X,Y):#producto, componente a componente, de los arrays X e Y
    Z=[]
    le=min(len(X),len(Y))
    for k in range(le):
        Z=Z+[X[k]*Y[k]]
    return(Z)

def fnorm(X):#suma las componentes del array X
    if X==[]:
        x=0
    else:
        x=X[0]
        for i in range(len(X)-1):
            x=x+X[i+1]
    return(x)

def SS(A,k):#suma las componentes k-ésimas de los arrays del array A
    s=0
    for i in range(len(A)):
        s=s+A[i][k]
    return(s)
    
def act(X,x):#multiplica el array X por el coeficiente x
    Y=[]
    for j in range(len(X)):
        Y=Y+[X[j]*x]
    return(Y)
    
def canonical(x,i,l):#genera el array canónico de longitud l y valor x en la posición i
    y=[]
    for j in range(l):
        if j==i:
            y=y+[x]
        else:
            y=y+[0]
    return(y)
    
def add(X,x,i):#añade un array canónico de valor x en la posición i al array X
    X=joint(X,canonical(x,i,len(X)))
    return(X)
    
def gadd(X,Y,J):#va añadiendo un array canónico de valor Y[j] en la posición J[j] al array X
    Z=X
    for j in range(len(J)):
        Z=add(Z,Y[j],J[j])
    return(Z)
    
def scanonical(x,i,l,k):#genera un array constante de valor k excepto x en la posición i
    y=[]
    for j in range(l):
        if j==i:
            y=y+[x]
        else:
            y=y+[k]
    return(y)
    
def sadd(X,x,i,k):#añade un array constante de valor k excepto x en la posición i al array X
    X=sumjoint(X,scanonical(x,i,len(X),k))
    return(X)
    
def sgadd(X,Y,J):#va añadiendo un array constante de valor X[0] excepto Y[j] en la posición J[j] al array X
    Z=X
    for j in range(len(J)):
        Z=sadd(Z,Y[j],J[j],X[0])
    return(Z)
    
def sumjoint(E,F):#suma los arrays de arrays E y F
    l=min(len(E),len(F))
    G=[]
    for i in range(l):
        G=G+[joint(E[i],F[i])]
    return(G)
    
def simpcomp(T,f):#aplica la función de una variable f a las componentes del array T
    D=[]
    for h in range(len(T)):
        D=D+[f(T[h])]
    return(D)
    
def doubcomp(T,f):#aplica recursivamente la función de dos variables f a cada par de componentes del array T
    D=T[0]
    if len(T)==1:
        D=D
    else:
        if len(T)==2:
            D=f(T[0],T[1])
        else:
            for h in range(len(T)-2):
                D=f(T[0],T[1])
                D=f(D,T[h+2])
    return(D)
    
def simpdoubcomp(C,S,f):#aplica la función de dos variables f al par de componentes i-ésimas de los arrays C y S
    l=min(len(C),len(S))
    P=[]
    for i in range(l):
        P=P+[f(C[i],S[i])]
    return(P)

def subarray(A,a,b):#extrae aquellos arrays del array A cuya componente a-ésima es b
    SUB=[]
    for l in range(len(A)):
        if A[l][a]==b:
            SUB=SUB+[A[l]]
        else:
            SUB=SUB
    return(SUB)

def dsubarray(A,T,b):#extrae aquellos arrays del array A cuya componente [T[0]][T[1]]-ésima es b
    DSUB=[]
    for l in range(len(A)):
        if A[l][T[0]][T[1]]==b:
            DSUB=DSUB+[A[l]]
        else:
            DSUB=DSUB
    return(DSUB)

def pre(A,i):#corta el array A desde la componente i en adelante
    B=[]
    for j in range(len(A)):
        if j<i:
            B=B+[A[j]]
        else:
            B=B
    return(B)

def post(A,i):#se come la primera parte del array A hasta la componente i
    B=[]
    for j in range(len(A)):
        if j>i:
            B=B+[A[j]]
        else:
            B=B
    return(B)

def retract(A,l):#se come los primeros elementos hasta el l de cada array del array de arrays A
    E=[]
    for i in range(len(A)):
        E=E+[post(A[i],l)]
    return E

def substract(A,i):#extrae la componente i del array A
    B=pre(A,i)+post(A,i)
    return(B)

def position(A,a):#array de las posiciones de la componente a en el array A
    J=[]
    for i in range(len(A)):
        if A[i]==a:
            J=J+[i]
        else:
            J=J
    return(J)

def superposition(A,B):#crea un array con las posiciones de las componentes del array B en el array A
    J=[]
    for i in range(len(B)):
        J=J+[position(A,B[i])]
    return(J)

def replace(Z,j,z):#reemplaza la componente j del array Z por z
    NZ=[]
    for k in range(len(Z)):
        if k==j:
            NZ=NZ+[z]
        else:
            NZ=NZ+[Z[k]]
    return(NZ)

def clean(Z):#elimina componentes repetidos del array Z
    V=[]
    for w in range(len(Z)):
        if len(position(Z,Z[w]))==1:
            V=V+[Z[w]]
        else:
            Z=replace(Z,w,[])
    return(joint(V))

def reindex(V,I):#permuta las componentes del array V según el array de índices I
    W=[]
    for i in range(len(I)):
        W=W+[V[I[i]]]
    return(W)

def order(X):#ordena el array X de menor a mayor
    maxX=max(X)
    RX=[]
    J=[]
    for i in range(len(X)):
        minX=min(X)
        Q=position(X,minX)
        RX=RX+[minX]
        X=substract(X,Q[0])
    return(RX)

def orderind(X):#saca la permutación de índices al ordenar el array X
    RX=order(X)
    Q=[]
    for i in range(len(X)):
        if (len(Q)<len(X)):
            P=position(X,RX[i])
            Q=Q+P
        else:
            Q=Q
    return(Q)

def ordercol(X,b):#ordena el array de arrays X por la columna b
    C=col(X,b)
    D=order(C)
    I=orderind(C)
    RX=reindex(X,I)
    OC=[RX,I]
    return(OC)
    
def superorder(X,b):#ordena el array de arrays X por la columna b
    Ba=col(col(X,6),b)
    O=order(Ba)[0]
    SP=superposition(Ba,O)
    In=clean(SP)
    RX=reindex(X,In)
    return(RX)

def tablita(rows,columnlabels,rowlabels):#hace una tabla con las filas rows con las etiquetas de columnas columnlabels y etiquetas de filas rowlabels
    data=np.array(rows)
    if rowlabels=='':
        df=pd.DataFrame(data,columns=columnlabels)
        with pd.option_context("display.max_rows",None,"display.max_columns",None):
            display(df)
    else:
        df=pd.DataFrame(data,columns=columnlabels,index=rowlabels)
        with pd.option_context("display.max_rows",None,"display.max_columns",None):
            display(df)
    return(df)

def tablitaS(rows,columnlabels,rowlabels):
    SP=[SS(rows,h) for h in range(len(columnlabels))]
    rows=rows+[SP]
    rowlabels=rowlabels+['TOTAL']
    tablita(rows,columnlabels,rowlabels)

def victor(X):#calcula las posiciones del máximo del array X
    M=max(X)
    P=position(X,max(X))
    return(P)

def committee(M,H,p):#devuelve un array de dos componentes que son variaciones de M y H de longitud p, respectivamente
    lm=len(M)
    lh=len(H)
    m=random.sample(range(0,lm),p)
    h=random.sample(range(0,lh),p)
    RM=[]
    for i in range(p):
        RM=RM+[M[m[i]]]
    RH=[]
    for i in range(p):
        RH=RH+[H[h[i]]]
    R=[RM,RH]
    return(R)
    
def compco(B,k,p):#devuelve un array donde cada componente tiene dos componentes de variaciones de longitud p de elementos de B
    S=[]
    for m in range(len(B)):
        S=S+[committee(B[m][k][0],B[m][k][1],p)]
    return(S)

ID=['DC4248','Profesorado Ayudante Doctor','Expresión Gráfica en la Ingeniería','Ingeniería Mecánica y Diseño Industrial','Escuela Superior de Ingeniería','Docencia en el área de conocimiento, preferentemente en materias vinculadas a las titulaciones impartidas en el centro de adscripción','Ingeniería en Diseño Industrial y Desarrollo del Producto','35 puntos']
CD=['','','']
EXM=['Ana Belén Ariza Villaverde','María del Pilar Carranza Cañadas','Elidia Beatriz Blázquez Parra','María Luz García Ceballos','Cristina Torrecillas Lozano','Laura García Ruesgas']
EXH=['Rafael Enrique Hidalgo Fernández','Guillermo Ortega Ruiz','Manuel Ignacio Bahamonde García','Oscar David de Cózar Macías','Francisco José Jiménez Hornero','Francisco Andrés Valderrama Gual']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DC4248=[ID,CD,EX,EXP]

ID=['DC5783','Profesorado Ayudante Doctor','Personalidad, Evaluación y Tratamiento Psicológico','Psicología','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento, preferentemente en materias vinculadas a las titulaciones impartidas en el centro de adscripción','Alteraciones neuropsicológicas en fibromialgia y su relación con variables clínicas, flujo sanguíneo cerebral e intervenciones focalizadas en la reducción del dolor','35 puntos']
CD=['','','']
EXM=['Beatriz Luengo López','Elisabeth Ruiz Padial','Lourdes Espinosa Fernández','Silvia Moreno Domínguez','María Nieves Pérez Marfil','María Isabel Peralta Ramírez']
EXH=['Manuel Isorna Folgar','Juan Verdejo Román','José Antonio Muela Martínez','Miguel Pérez García','Manuel Jesús Ruiz Muñoz','Rafael Tomás Andújar Barroso']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DC5783=[ID,CD,EX,EXP]

ID=['DCXXXX','Profesorado Ayudante Doctor','','','','Docencia en el área de conocimiento, preferentemente en materias vinculadas a las titulaciones impartidas en el centro de adscripción','','35 puntos']
CD=['','','']
EXM=['','','','','','']
EXH=['','','','','','']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DCXXXX=[ID,CD,EX,EXP]

DCPAD=[DCXXXX]

ID=['DC5716','Profesorado Permanente Laboral/Profesorado Contratado Doctor (Marie Curie)','Ciencia de los Materiales e Ingeniería Metalúrgica','Ciencia de los Materiales e Ingeniería Metalúrgica y Química Inorgánica','Escuela Técnica Superior de Ingeniería de Algeciras','Docencia en el área de conocimiento, preferentemente en materias vinculadas a las titulaciones impartidas en el centro de adscripción','Desarrollo de materiales poliméricos sostenibles para fabricación aditiva mediante metodologías computacionales','50 puntos']
CD=['','','']
EXM=['María Dolores La Rubia García','Ana María Beltrán Custodio','Rocío Ruiz Bustos','Silvina Cerveny Murcia','Marián A. Gómez-Fatou Rodríguez','Dolores Eliche Quesada']
EXH=['Jesús Hernández Saz','Juan Gabriel Lozano Suárez','Joost Van Duijn','Juan Rodríguez Hernández','Gustavo Ariel Schwartz Pomeraniec','Miguel Ángel López Manchado']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DC5716=[ID,CD,EX,EXP]

ID=['DC5717','Profesorado Permanente Laboral/Profesorado Contratado Doctor (María Zambrano)','Zoología','Biología','Facultad de Ciencias del Mar y Ambientales','Docencia en el área de conocimiento, preferentemente en materias vinculadas a las titulaciones impartidas en el centro de adscripción','Fisiología digestiva de especies marinas','50 puntos']
CD=['','','']
EXM=['Ana Bolaños Martín','María Soledad Izquierdo López','Amalia Morales Hernández','Marta Pola Pérez','Lidia Esther Robaina Robaina','Macarena Ros Clemente']
EXH=['Francisco Javier Alarcón López','Luis José Fernández Rodríguez','Félix Hidalgo Puertas','Tomás Francisco Martínez Moya','Daniel Montero Vítores','Carlos Valle Pérez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DC5717=[ID,CD,EX,EXP]

ID=['DCYYYY','Profesorado Permanente Laboral/Profesorado Contratado Doctor','','','','Docencia en el área de conocimiento, preferentemente en materias vinculadas a las titulaciones impartidas en el centro de adscripción','','50 puntos']
CD=['','','']
EXM=['','','','','','']
EXH=['','','','','','']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DCYYYY=[ID,CD,EX,EXP]

DCPPL=[DCYYYY]

DC=DCPPL+DCPAD
DCS=extract(extract(DC))

ID=['DF5223','Profesoras y Profesores Titulares de Universidad. Categoría asistencial: Facultativo Especialista de área','Microbiología','Ciencias de la Salud','Biomedicina, Biotecnología y Salud Pública','Facultad de Medicina. Centro asistencial: Hospital Universitario Puerta del Mar','Docencia en el área de conocimiento. Especialidad/perfil asistencial: Microbiología','Bacterias gram negativas multirresistentes']
CD=['','','']
EXM=['Patricia Carmen Muñoz García','María Francisca Colom Valiente','Concepción Gimeno Cardona','Elena Eraso Barrio','Marta Hernández Pérez','Cristina Pilar Seral García']
EXH=['Jordi Vila Estapé','José María Eiros Bouza','Guillermo Quindós Andrés','David Navarro Ortega','Roberto Alfonso Alonso Fernández','Luis Martínez Martínez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5223=[ID,CD,EX,EXP]

ID=['DF5686','Profesoras y Profesores Titulares de Universidad (Emergia)','Mecánica de Fluidos','Ingeniería y Arquitectura','Ingeniería Mecánica y Diseño Industrial','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Investigación en el área de conocimiento, preferentemente en simulación computacional a gran escala']
CD=['','','']
EXM=['Elena Beatriz Martín Ortega','María Inmaculada Iglesias Estradé','María Rocío Bolaños Jiménez','Anne Marie Elisabeth Gosset','María Guadalupe Cabezas Martín','Paloma Gutiérrez Castillo']
EXH=['José Manuel Gordillo Arias de Saavedra','Guillaume Maurice Riboux Acher','Ignacio González Loscertales','Luis Parras Anguita','Jesús Carlos Martínez Bazán','Francisco Javier Rodríguez Rodríguez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5686=[ID,CD,EX,EXP]

ID=['DF5723','Profesoras y Profesores Titulares de Universidad','Análisis Geográfico Regional','Arte y Humanidades','Historia, Geografía y Filosofía',' Facultad de Filosofía y Letras','Docencia en el área de conocimiento',' Planificación y gestión en áreas costero-marinas']
CD=['','','']
EXM=['Gemma Canoves Valiente','María José Cuesta Aguilar','Juana María Petrus Bey','Julia Salom Carrasco','Susana Rosa Navarro Rodríguez','Yolanda Jiménez Olivencia']
EXH=['Francisco José Torres Alfosea','José Fernando Vera Rebollo','Juan Antonio Márquez Domínguez','Joaquín Farinos Dasi','Enrique Javier López Lara','Enrique Luis Santos Pavón']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5723=[ID,CD,EX,EXP]

ID=['DF5724','Profesoras y Profesores Titulares de Universidad','Análisis Geográfico Regional','Arte y Humanidades','Historia, Geografía y Filosofía','Facultad de Ciencias Sociales y de la Comunicación','Docencia en el área de conocimiento','Planificación y gestión en áreas costero-marinas con énfasis en Iberoamérica']
CD=['','','']
EXM=['Gemma Canoves Valiente','María José Cuesta Aguilar','Juana María Petrus Bey','Julia Salom Carrasco','Susana Rosa Navarro Rodríguez','Yolanda Jiménez Olivencia']
EXH=['Francisco José Torres Alfosea','José Fernando Vera Rebollo','Juan Antonio Márquez Domínguez','Joaquín Farinos Dasi','Enrique Javier López Lara','Enrique Luis Santos Pavón']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5724=[ID,CD,EX,EXP]

ID=['DF5725','Profesoras y Profesores Titulares de Universidad','Bioquímica y Biología Molecular','Ciencias','Biomedicina, Biotecnología y Salud Pública','Facultad de Medicina','Docencia en el área de conocimiento',' Bioquímica de las infecciones víricas crónicas']
CD=['','','']
EXM=['Carmen María Michán Doña','María Dolores Girón González','Guadalupe Gómez Baena','Nerea Osinal de Moraleja','Carolina Gómez Llorente','María Isabel Muñoz Barroso']
EXH=['Rafael Salto González','Luis Fontana Gallego','Jesús Valentín Jorrín Novo','Antonio Ayala Gómez','Antonio Romero Ruiz','Francisco Manuel Vega Moreno']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5725=[ID,CD,EX,EXP]

ID=['DF5726','Profesoras y Profesores Titulares de Universidad','Ciencia de los Materiales e Ingeniería Metalúrgica','Ingeniería y Arquitectura','Ciencia de los Materiales e Ingeniería Metalúrgica y Química Inorgánica','Facultad de Ciencias','Docencia en el área de conocimiento','Ciencia e ingeniería de materiales poliméricos para fabricación aditiva y superficies funcionales']
CD=['','','']
EXM=['Ana María Beltrán Custodio','Eva María Pérez Soriano','Rocío Ruiz Bustos','Marián Gómez-Fatou Rodríguez','Alexandra Muñoz Bonilla','Dolores Eliche Quesada']
EXH=['Juan Gabriel Lozano Suárez','Joost Van Dujin','Juan Rodríguez Hernández','Juan Salvador Bueno Rodríguez','Alberto Gallardo Ruiz','Daniel López García']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5726=[ID,CD,EX,EXP]

ID=['DF5727','Profesoras y Profesores Titulares de Universidad','Comercialización e Investigación de Mercados','Ciencias Sociales y Jurídicas','Marketing y Comunicación','Facultad de Ciencias Sociales y de la Comunicación','Docencia en el área de conocimiento','Marketing turístico sostenible en modalidades turísticas emergentes turismo halal y experiencial']
CD=['','','']
EXM=['Eva María Murgado Armenteros','María Gutiérrez Salcedo','Manuela Vega Zamora','Ana María Campón Cerro','María del Pilar Alarcón Urbistondo','Ana María Casado Molina']
EXH=['Juan Miguel Rey Pino','Manuel Parras Rosa','Francisco José Torres Ruiz','José Manuel Hernández Mogollón','Carlos Javier Rodríguez Rad','Rafael Anaya Sánchez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5727=[ID,CD,EX,EXP]

ID=['DF5728','Profesoras y Profesores Titulares de Universidad','Comercialización e Investigación de Mercados','Ciencias Sociales y Jurídicas','Marketing y Comunicación','Facultad de Ciencias Sociales y de la Comunicación','Docencia en el área de conocimiento','Análisis de la efectividad publicitaria desde la perspectiva del neuromarketing']
CD=['','','']
EXM=['María Belén Prados Peña','Lucía Porcu','Ana María Casado Molina','Eva María Murgado Armenteros','Pilar Alarcón Urbistondo','María Gutiérrez Salcedo']
EXH=['Juan Miguel Rey Pino','Julio Vena Oya','Francisco José Torres Ruiz','Francisco Javier Montoro Ríos','Luis Doña Toledo','Sebastián Molinillo Jiménez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5728=[ID,CD,EX,EXP]

ID=['DF5729','Profesoras y Profesores Titulares de Universidad','Derecho Civil','Ciencias Sociales y Jurídicas','Derecho Privado','Facultad de Derecho','Docencia en el área de conocimiento','Nuevas Perspectivas del Derecho de Contratos y de Daños']
CD=['','','']
EXM=['María Belén Sáinz-Cantero Caparrós', 'Josefina Alventosa del Río','María Elena Cobas Cobiella','Asunción Marín Velarde','Raquel Guillem Catalán','Iciar Cordero Cutillas']
EXH=['José Ramón Verda Beamonte','Pedro Chaparro Matamoros','Eduardo Vázquez de Castro','Javier Martínez Calvo','Ángel Sánchez Hernández','Rafael Colina Garea']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5729=[ID,CD,EX,EXP]

ID=['DF5730','Profesoras y Profesores Titulares de Universidad','Derecho Internacional Privado','Ciencias Sociales y Jurídicas','Derecho Internacional Público, Penal y Procesal','Facultad de Derecho','Docencia en el área de conocimiento','Nuevas Tecnologías']
CD=['','','']
EXM=['Irene Blázquez Rodríguez','Laura Isabel García Álvarez','María Pilar Diago Diago','Beatriz Campuzano Díaz','Carmen Otero García Castrillón','Ángeles Lara Aguado']
EXH=['Alfonso Ybarra Bores','Alfonso Ortega Giménez','Ricardo Rueda Valdivia','Jacinto Pablo Quinzá Redondo','Josep Gunnar Horrach Armo','Enrique Fernández Masia']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5730=[ID,CD,EX,EXP]

ID=['DF5731','Profesoras y Profesores Titulares de Universidad','Derecho Mercantil','Ciencias Sociales y Jurídicas','Derecho Mercantil','Facultad de Derecho (Sede Algeciras)','Docencia en el área de conocimiento','Derecho marítimo']
CD=['','','']
EXM=['María Jesús Guerrero Lebrón','Josefa Brenes Cortés','Angélica Díaz de la Rosa','Raquel López Ortega','María de las Nieves López Santana','Lucía Alvarado Herrera']
EXH=['Pedro Jesús Baena Baena','Rafael La Casa García','Pablo Martínez-Gijón Machuca','Jose Carlos Vázquez Cueto','Alfonso Rodríguez de Quiñones de Torres','Carlos Górriz López']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5731=[ID,CD,EX,EXP]

ID=['DF5732','Profesoras y Profesores Titulares de Universidad','Derecho Penal','Ciencias Sociales y Jurídicas','Derecho Internacional Público, Penal y Procesal','Facultad de Derecho','Docencia en el área de conocimiento','Investigación en el Área de Derecho Penal']
CD=['','','']
EXM=['Esther Rebollo Vargas','María Inmaculada Valeije Álvarez','María Paz Lloira García','María Pilar Otero González','Miriam Cugat Mauri','Paz Mercedes de la Cuesta Aguado']
EXH=['Rafael Rebollo Vargas','Jacobo Dopico Gómez-Aller','Eduardo Ramón Ribas','Joan Baucells i Lladós','Fernando Antonio Vázquez-Portomeñe Seijas','Guillermo Portilla Contreras']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5732=[ID,CD,EX,EXP]

ID=['DF5733','Profesoras y Profesores Titulares de Universidad','Derecho Procesal','Ciencias Sociales y Jurídicas','Derecho Internacional Público, Penal y Procesal','Facultad de Derecho','Docencia en el área de conocimiento','Derecho Procesal']
CD=['','','']
EXM=['María del Corral Arangüena Fanego','Silvia Barona Vilar','María Inmaculada Sánchez Barrios','Rosa Rodríguez Bahamonde','María Isabel Hernández Gómez','Olga Fuentes Soriano']
EXH=['Federico Bueno de Mata','Fernando Martín Diz','Manuel Richard González','Jesús Miguel Hernández Galilea','Francisco López Simó','Xulio José Ferreiro Baamonde']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5733=[ID,CD,EX,EXP]

ID=['DF5734','Profesoras y Profesores Titulares de Universidad','Didáctica de la Lengua y la Literatura','Ciencias Sociales y Jurídicas','Didáctica de la Lengua y la Literatura','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Formación docente en lectura no literaria y mediación tecnológica en DLL (L1)']
CD=['','','']
EXM=['Cristina Milagrosa Castillo Rodríguez','María Victoria López Pérez','María Nieves Martín Rogero','María Guadalupe de la Maya Retamar',' María Pilar Núñez Delgado','Mónica Ruiz Bañuls']
EXH=['Santiago Fabregat Barrios','Jesús García Laborda','Alberto Gutiérrez Gil','José Luis Ortega Martín','Agustín Reyes Torres','José Soto Vázquez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5734=[ID,CD,EX,EXP]

ID=['DF5735','Profesoras y Profesores Titulares de Universidad','Didáctica de la Lengua y la Literatura','Ciencias Sociales y Jurídicas','Didáctica de la Lengua y la Literatura','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Formación docente para la enseñanza adquisitiva del inglés como lengua extranjera']
CD=['','','']
EXM=['María Dolores Alonso-Cortés Fradejas','Beatriz Cortina Pérez','María Fernández Agüero','María del Pilar García Carcedo','María Dolores García Pastor','María Felicidad Tabuenca Cuevas']
EXH=['Jesús García Laborda','Fernando Guzmán Simón','José Luis Ortega Martín','Ramón Pérez Parejo','Raúl Ruiz Cecilia','Jose Soto Vázquez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5735=[ID,CD,EX,EXP]

ID=['DF5736','Profesoras y Profesores Titulares de Universidad','Didática de la Matemática','Ciencias Sociales y Jurídicas','Didáctica','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Entornos lúdico-tecnológicos en la formación inicial de maestros/as de matemáticas']
CD=['','','']
EXM=['Raquel Fernández Cezar','Isabel María Romero Albadalejo','María Teresa Sánchez Compaña','María Teresa Sanz García','Silvia Martínez Sanahuja','Marta Pérez Rodríguez']
EXH=['Juan Miguel Belmonte Gómez','Gustavo Raúl Canadas de la Fuente','Tomás Ángel Sierra Delgado','Manuel Torralbo Rodríguez','Jose Antonio González-Calero Somoza','Antonio Codina Sánchez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5736=[ID,CD,EX,EXP]

ID=['DF5737','Profesoras y Profesores Titulares de Universidad','Didáctica y Organización Escolar','Ciencias Sociales y Jurídicas','Didáctica','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Políticas y prácticas socioeducativas en Educación Inclusiva']
CD=['','','']
EXM=['María Ángeles Parrilla Latas','María Dolores Forteza Forteza','María Begoña Vigo Arrazola','Leire Darretxe Urrutxi','Francisca González Gil','Analia Elisabeth Leite Méndez']
EXH=['José Ignacio Rivas Flores','Alejandro Rodríguez Martín','Ignacio Haya Salmón','Emilio Álvarez Arregui','Joxe Amiama Iberguren','Luis Mariano Torrego Egido']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5737=[ID,CD,EX,EXP]

ID=['DF5738','Profesoras y Profesores Titulares de Universidad','Derecho Romano','Ciencias Sociales y Jurídicas','Derecho Privado','Facultad de Derecho','Docencia en el área de conocimiento','Derecho de Propiedad y Propiedades Claudicantes en Derecho Romano']
CD=['','','']
EXM=['Patricia Panero Oria','María del Carmen Lázaro Guillamón','Amparo Montañana Casaní','Aurora López Güeto','Alicia Valmaña Ochaita','Margarita Fuenteseca Degeneffe']
EXH=['Santiago Castán Pérez-Gómez','Francisco José Cuena Boy','Bernardo Jesús Periñán Gómez','Ramón Herrera Bravo','Ramón Perfecto Rodríguez Montero','Xesús Pérez López']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5738=[ID,CD,EX,EXP]

ID=['DF5739','Profesoras y Profesores Titulares de Universidad','Economía Aplicada','Ciencias Sociales y Jurídicas','Economía General','Facultad de Ciencias Sociales y de la Comunicación','Docencia en el área de conocimiento','Economía del Turismo y Agroalimentación']
CD=['','','']
EXM=['Elena María Barcena Martín','Eva María Buitrago Esquinas','María Jesús Delgado Rodríguez','María Carmen Lizarraga Mollinedo','Encarnación Moral Pajares','María Teresa Sánchez Martínez']
EXH=['Alejando Francisco García Pozo','Fernando José García Quero','Alfredo José Mainar Causape','José Miguel Martínez Paz','Juan Carlos Morán Álvarez','Juan Carlos Rodríguez Cohard']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5739=[ID,CD,EX,EXP]

ID=['DF5740','Profesoras y Profesores Titulares de Universidad','Educación Física y Deportiva','Ciencias Sociales y Jurídicas','Didáctica de la Educación Física, Plástica y Musical','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Condición Física y Riesgo Cardiometabólico en Niños']
CD=['','','']
EXM=['Susana Aznar Laín','María Isabel Piñar López','Rocío Cupeiro Coto','Cecilia Dorado García','Yaira María Barranco Ruiz','María del Pilar Sainz de Baranda Andújar']
EXH=['Alberto Nuviala Nuviala','Narcís Gusi Fuertes','Francisco de Borja Sañudo Corrales','Manuel Delgado Fernández','Rafael Timón Andrada','Luis Javier Chirosa Ríos']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5740=[ID,CD,EX,EXP]

ID=['DF5741','Profesoras y Profesores Titulares de Universidad','Educación Física y Deportiva','Ciencias Sociales y Jurídicas','Didáctica de la Educación Física, Plástica y Musica','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Investigación en Deportistas de Élite de Vela Ligera']
CD=['','','']
EXM=['Susana Aznar Laín','Marta García Tascón','Palma Chillón Garzón','Ana María Gómez Tafalla','Leonor Gallardo Guerrero','Yaira María Barranco Ruiz']
EXH=['Ángel Luis Pérez Pueyo','Francisco de Borja Sañudo Corrales','Alberto Nuviala Nuviala','Juan José Salinero Martín','Daniel Juárez Santos-García','Narcís Gusi Fuertes']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5741=[ID,CD,EX,EXP]

ID=['DF5742','Profesoras y Profesores Titulares de Universidad','Enfermería','Ciencias de la Salud','Enfermería y Fisioterapia','Facultad de Enfermería y Fisioterapia','Docencia en el área de conocimiento','Metodología enfermera y lenguajes enfermeros estandarizados. Desarrollo y validación psicométrica/clinimétrica de instrumentos de medida en cuidados de salud']
CD=['','','']
EXM=['Bárbara Badanta Romero','Rocío de Diego Cordero','María Josefa Cabañero Martínez','Eva María Gabaldón Bravo','Silvia Corchón Arreche','María Luisa Estarlich Estarlich']
EXH=['Manuel Vaquero Abellán','José Luis Gómez Urquiza','Raúl Juárez Vela','César Hueso Montoro','Manuel Pabón Carrasco','Rubén Morilla Romero de la Osa']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5742=[ID,CD,EX,EXP]

ID=['DF5743','Profesoras y Profesores Titulares de Universidad','Enfermería','Ciencias de la Salud','Enfermería y Fisioterapia','Facultad de Enfermería y Fisioterapia','Docencia en el área de conocimiento','Alfabetización en Salud para poblaciones diversas en el ámbito de la Enfermería']
CD=['','','']
EXM=['María Luisa Estarlich Estarlich','Silvia Corchón Arreche','Bárbara Bandata Romero','Esperanza Begoña García Navarro','Adelina Martín Salvador','Elia Fernández Martínez']
EXH=['José Luis Gómez Urquiza','Rafael Arcángel Caparrós González','Sergio Barrientos Trigo','Antonio Martínez Sabater','Raúl Juárez Vela','Manuel Vaquero Abellán']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5743=[ID,CD,EX,EXP]

ID=['DF5745','Profesoras y Profesores Titulares de Universidad','Estadística e Investigación Operativa','Ingeniería y Arquitectura','Estadística e Investigación Operativa','Facultad de Ciencias Económicas y Empresariales','Docencia en el área de conocimiento','Estadística aplicada al tratamiento de datos de estaciones GNSS y al establecimiento de modelos de deformación superficial']
CD=['','','']
EXM=['María del Carmen Gale Pola','Mercedes Landete Ruiz','María Teresa León Mendoza','Ana Meca Martínez','María Teresa Ortuño Sánchez','Inmaculada Rodríguez Martín']
EXH=['Pascual Fernández Hernández','José Miguel Gutiérrez Expósito','Francisco Javier Martín Campo','Juan Francisco Monge Ivars','Juan José Peiró Ramada','Juan José Salazar González']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5745=[ID,CD,EX,EXP]

ID=['DF5746','Profesoras y Profesores Titulares de Universidad','Expresión Gráfica en la Ingeniería','Ingeniería y Arquitectura','Ingeniería Mecánica y Diseño Industrial','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Diseño industrial y desarrollo de productos para tecnologías de fabricación aditiva de gran formato']
CD=['','','']
EXM=['Cristina Martín Doñate','Ana Belén Villaverde Ariza','Cristina Torrecillas Lozano','María Luz García Ceballos','Amparo Jordá Vilaplana','Silvia Sempere Ripoll']
EXH=['Ismael Lengua Lengua','Oscar David de Cozar Macías','Rafael Enrique Hidalgo Fernández','Antonio Fernández Álvarez','Miguel Castro García','Isidro Ladrón de Guevara López']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5746=[ID,CD,EX,EXP]

ID=['DF5747','Profesoras y Profesores Titulares de Universidad','Expresión Gráfica en la Ingeniería','Ingeniería y Arquitectura','Ingeniería Mecánica y Diseño Industrial','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Integración de entornos de realidad virtual y registro neurofisiológico en la evaluación del diseño']
CD=['','','']
EXM=['María Jesús Agost Torres','María Carmen González Lluch','Beatriz Defez García','Larisa Dunai Dunai','Ana Belén Ariza Villaverde','Norena Natalia Martín Dorta']
EXH=['Ignacio Tortajada Montañana','Jose Pablo Suárez Rivero','Rafael Enrique Hidalgo Fernández','Pedro Pablo Company Calleja','José Luis Saorin Pérez','Jorge Martín Gutiérrez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5747=[ID,CD,EX,EXP]

ID=['DF5748','Profesoras y Profesores Titulares de Universidad','Filología Francesa','Arte y Humanidades','Filología Francesa e Inglesa','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','La influencia de Jean Lorrain entre sus contemporáneos']
CD=['','','']
EXM=['Lourdes Ángeles Terrón Barbosa','Mercedes Montoro Araque','Adela Cortijo Talavera','Encarnación Medina Arjona','María Inmaculada Illanes Ortega','Eva Pich Ponce']
EXH=['Domingo Pujante González','André Benard Bénit','Pedro Salvador Méndez Robles','Justo Bolekia Boleká','Rafael Ruiz Álvarez','Tomás Gonzalo Santos']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5748=[ID,CD,EX,EXP]

ID=['DF5749','Profesoras y Profesores Titulares de Universidad','Filología Francesa','Arte y Humanidades','Filología Francesa e Inglesa','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Emergencia de la literatura negro-africana en el contexto colonial francés']
CD=['','','']
EXM=['María Lourdes Carriedo López','Dulce María González Doreste','María Carmen Figuerola Cabrol','Ana Labra Cenitagoya','Eva María Pich Ponce','María Lidia Vázquez Jiménez']
EXH=['Ignacio Iñarrea Heras','Jorge Augusto Luengo López','Pedro Salvador Méndez Robles','Domingo Pujante González','Ignacio Ramos Gay','Rafael Ruiz Álvarez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5749=[ID,CD,EX,EXP]

ID=['DF5750','Profesoras y Profesores Titulares de Universidad','Filología Inglesa','Arte y Humanidades','Filología Francesa e Inglesa','Escuela de Ingenierías Marina, Naútica y Radioelectrónica','Docencia en el área de conocimiento','Enseñanza-aprendizaje de inglés marítimo y terminología de la navegación y el transporte marítimo en inglés']
CD=['','','']
EXM=['Rosa Rabadán Álvarez','María Luisa Carrió Pastor','Cristina Tejedor Martínez','Florentina Micaela Mena Martínez','Ingrid Cobos López','María del Carmen Balbuena Torezano']
EXH=['Juan Manuel Hernández Campoy','Javier Pérez Guerra','Miguel Ángel Candel Mora','Antonio García Gómez','Juan Antonio Cutillas Espinosa','Pedro Joaquín Mogorrón Huerta']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5750=[ID,CD,EX,EXP]

ID=['DF5751','Profesoras y Profesores Titulares de Universidad','Fisiología','Ciencias y Ciencias de la Salud','Biomedicina, Biotecnología y Salud Pública','Facultad de Medicina','Docencia en el área de conocimiento','Fisiopatología molecular y regeneración celular en el sistema nervioso y vascular']
CD=['','','']
EXM=['María Luisa Calonge Castrillo','Miriam Echevarría Irusta','María Mercedes Cano Rodríguez','María Reposo Ramírez Lorca','Raquel del Toro Estévez','María José Peral Rubio']
EXH=['Juan Casto Rivadulla Fernández','Ricardo Pardal Redondo','Tarik Smani Hajami','Alfonso Mate Barrero','Francisco Javier Villadiego Luque','Pablo García Miranda']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5751=[ID,CD,EX,EXP]

ID=['DF5752','Profesoras y Profesores Titulares de Universidad','Fisioterapia','Ciencias de la Salud','Enfermería y Fisioterapia','Facultad de Enfermería y Fisioterapia','Docencia en el área de conocimiento','Investigación clínica mediante electrólisis percutánea intratisular y presión negativa pulsada']
CD=['','','']
EXM=['Guadalupe Molina Torres','Noelia Zagalaz Anula','María Torres Lacomba','Noelia Galiano Castillo','Noelia Moreno Morales','Yolanda Castellote Caballero']
EXH=['Manuel Romero Saldaña','Manuel González Sánchez','José Jesús Jiménez Rejano','Agustín Aibar Almazán','Sebastián Sanz Martos','Francisco Javier Molina Ortega']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5752=[ID,CD,EX,EXP]

ID=['DF5753','Profesoras y Profesores Titulares de Universidad','Genética','Ciencias','Biomedicina, Biotecnología y Salud Pública','Facultad de Ciencias','Docencia en el área de conocimiento','Investigación en el Área de conocimiento, preferentemente en Genética y Genómica en organismos marinos']
CD=['','','']
EXM=['Vanesa Valdiglesias García','Mónica Bullejos Martín','Ana Grande Pérez','Julia Bejar Alvarado','María Teresa González Jaén','Francisca Robles Rodríguez']
EXH=['Antonio Carvajal Rodríguez','Yaisel Juan Borrell Pichs','Miguel Burgos Poyatos','Pedro García García','Cayo Juan Ramos Rodríguez','Andrés Martínez Lage']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5753=[ID,CD,EX,EXP]

ID=['DF5754','Profesoras y Profesores Titulares de Universidad','Genética','Ciencias','Biomedicina, Biotecnología y Salud Pública','Facultad de Ciencias','Docencia en el área de conocimiento','Genética y Genómica de Organismos Marinos']
CD=['','','']
EXM=['María Teresa González Jaén','Mónica Bullejos Martín','Ana Grande Pérez','Julia Béjar Alvarado','María Belén Gómez Pardo','María Paloma Morán Martínez']
EXH=['Andrés Martínez Lage','Antonio Sánchez Baca','Luis Enrique Sáenz de Miera Carnicer','Juan Capel Salinas','Miguel Burgos Poyatos','Cayo Juan Ramos Rodríguez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5754=[ID,CD,EX,EXP]

ID=['DF5755','Profesoras y Profesores Titulares de Universidad','Historia de América','Arte y Humanidades','Historia Moderna, Comtemporánea, de América y del Arte','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','El sistema defensivo atlántico de la Monarquía española: de las Antillas Mayores a Cádiz, siglo XVIII']
CD=['','','']
EXM=['Rosario Márquez Macías','María Eugenia Petit-Breuilh Sepúlveda','María Pilar Cagiao Vila','Izaskun Álvarez Cuartero','María Palmira Vélez Jiménez','Lucía Provencio Garrigós']
EXH=['Juan José Sánchez Baena','José Manuel Santos Pérez','Sigfrido Vázquez Cienfuegos','Jesús Adanez Pavón','Eduardo Rey Tristán','Alberto Baena Zapatero']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5755=[ID,CD,EX,EXP]

ID=['DF5756','Profesoras y Profesores Titulares de Universidad','Ingeniería de los Procesos de Fabricación','Ingeniería y Arquitectura','Ingeniería Mecánica y Diseño Industrial','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Texturizado láser y tribología aplicada a ingeniería de fabricación']
CD=['','','']
EXM=['María Rosario Domingo Navas','Ana Isabel Fernández Abia','María Luisa García-Romeu de Luna','Piera Maresca','María Dolores Marrero Alemán','Eva María Rubio Alvir']
EXH=['Juan José Aguilar Martín','Juan Antonio García Manrique','Aitzol Lamínik Mentxaca','Pedro José Núñez López','Lorenzo Sevilla Hurtado','Carpóforo Vallellano Martín']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5756=[ID,CD,EX,EXP]

ID=['DF5757','Profesoras y Profesores Titulares de Universidad','Ingeniería de Sistemas y Automática','Ingenería y Arquitectura','Ingeniería en Automática, Electrónica, Arquitectura y Redes de Computadores','Escuela Técnica Superior de Ingeniería de Algeciras','Docencia en el área de conocimiento','Investigación en modelado y control de sistemas híbridos de energías renovables con almacenamiento de energía e hidrógeno']
CD=['','','']
EXM=['Ana María Cruz Martín','Isabel García Morales','María del Carmen Meizoso López','María Belén Curto Diego','Inés Tejado Balsera','Mónica Ballesta Galdeano']
EXH=['Ángel Gaspar González Rodríguez','Carlos Vivas Venegas','Teodoro Rafael Álamo Cantarero','Manuel Berenguel Soria','José Luis Calvo Rolle',' Fernández Lozano']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5757=[ID,CD,EX,EXP]

ID=['DF5758','Profesoras y Profesores Titulares de Universidad','Ingeniería de Sistemas y Automática','Ingenería y Arquitectura','Ingeniería en Automática, Electrónica, Arquitectura y Redes de Computadores','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Investigación en telemedicina de pacientes con enfermedades pulmonares']
CD=['','','']
EXM=['Ana María Cruz Martín','Ascensión Zafra Cabeza','Isabel García Morales','María Ángeles Malfaz Vázquez','María del Carmen Meizoso López',' Guijarro Mata-García']
EXH=['Ángel Gaspar González Rodríguez','Daniel Limón Marruedo','Fernando Caballero Benítez','José Luis Calvo Rolle','Manuel Ruiz Arahal','Teodoro Rafael Álamo Cantarero']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5758=[ID,CD,EX,EXP]

ID=['DF5759','Profesoras y Profesores Titulares de Universidad','Ingeniería e Infraestructura de los Transportes','Ingeniería y Arquitectura','Ingeniería Industrial e Ingeniería Civil','Escuela Técnica Superior de Ingeniería de Algeciras','Docencia en el área de conocimiento','Nuevas tendencias de modelado de sistemas logísticos y de transporte en entornos marítimo-portuarios. Aplicaciones de machine learning en este campo']
CD=['','','']
EXM=['María del Pilar Jiménez Gómez','Griselda López Maldonado','Ana María Rivas Álvarez','Marta Rojo Arce','Laura Garach Morcillo','Begoña Guirao Abad']
EXH=['Alberto Camarero Orive','Luis Aragonés Pomares','Hernán Gonzalo Orden','Armando Ortuño Padilla','José Luis Moura Berodia','Santos Sánchez-Cambronero García-Moreno']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5759=[ID,CD,EX,EXP]

ID=['DF5760','Profesoras y Profesores Titulares de Universidad','Lengua Española','Arte y Humanidades','Filología','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Sociolingüística histórica del español']
CD=['','','']
EXM=['María del Mar Espejo Muriel','Livia Cristina García Aguiar','Ángeles García Aranda','María del Carmen Hernández González','María Agueda Moreno Moreno','Mercedes Quilis Merín']
EXH=['Ambrosio Miguel Calderón Campos','Francisco Manuel Carriscondo Esquivel','Andrés Enrique Arias','Alberto Hernando García-Cervigón','Miguel Ángel Puche Lorenzo','Daniel Moisés Sáez Rivera']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5760=[ID,CD,EX,EXP]

ID=['DF5761','Profesoras y Profesores Titulares de Universidad','Lingüística General','Arte y Humanidades','Filología','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Morfología Léxica']
CD=['','','']
EXM=['Beatriz Gallardo Paúls','María del Carmen Fernández Juncal','Carmen Galán Rodríguez','Gabriela Prego Vázquez','Sonia Madrid Cánovas','Ana María Cestero Mancera']
EXH=['Ricardo Morant Marco','Francisco José Salguero Lamillar','Alfonso Zamorano Aguilar','Francisco de Asís Fernández García','Carlos Hernández Sacristán','Enrique Nicolás Serra Alegre']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5761=[ID,CD,EX,EXP]

ID=['DF5762','Profesoras y Profesores Titulares de Universidad','Lingüística General','Arte y Humanidades','Filología','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Lingüística contrastiva aplicada al discurso']
CD=['','','']
EXM=['Beatriz Gallardo Paúls','María del Carmen Fernández Juncal','Carmen Galán Rodríguez','Gabriela Prego Vázquez','Sonia Madrid Cánovas','Ana María Cestero Mancera']
EXH=['Ricardo Morant Marco','Francisco José Salguero Lamillar','Alfonso Zamorano Aguilar','Francisco de Asís Fernández García','Carlos Hernández Sacristán','Enrique Nicolás Serra Alegre']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5762=[ID,CD,EX,EXP]

ID=['DF5763','Profesoras y Profesores Titulares de Universidad','Organización de Emoresas','Ciencias Sociales y Jurídicas','Organización de Empresas','Facultad de Ciencias Económicas y Empresariales','Docencia en el área de conocimiento','Gestión del Capital Intelectual en Equipos Científicos']
CD=['','','']
EXM=['Susana Pasamar Reyes','Rosalía Díaz Carrión','María Eugenia Sánchez Vidal','Mónica Santana Hernández','María del Pilar Jerez Gómez','Raquel Sanz Valle']
EXH=['Álvaro López Cabrales','Alejandro Escribá Esteve','Joaquín Alegre Vidal','Gregorio Sánchez Marín','José Luis Galán González','Luigi Stirpe']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5763=[ID,CD,EX,EXP]

ID=['DF5764','Profesoras y Profesores Titulares de Universidad','Organización de Emoresas','Ciencias Sociales y Jurídicas','Organización de Empresas','Facultad de Ciencias Económicas y Empresariales','Docencia en el área de conocimiento','Gestión del Capital Intelectual']
CD=['','','']
EXM=['Carmen Cabello Medina','María del Pilar Jerez Gómez','María Eugenia Sánchez Vidal','Susana Pasamar Reyes','Mónica Santana Hernández','Rosalía Díaz Carrión']
EXH=['Álvaro López Cabrales','Alejandro Escribá Esteve','Luigi Stirpe','Gregorio Sánchez Marín','Joaquín Alegre Vidal','José Luis Galán González']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5764=[ID,CD,EX,EXP]

ID=['DF5765','Profesoras y Profesores Titulares de Universidad','Organización de Emoresas','Ciencias Sociales y Jurídicas','Organización de Empresas','Facultad de Ciencias Sociales y de la Comunicación','Docencia en el área de conocimiento','Control y Medida de la Eficiencia']
CD=['','','']
EXM=['Rosalía Díaz Carrión','Carmen Cabello Medina','Ana María Bojica Bojica','Lidia Andrades Caldito','Encarnación García Sánchez','Nuria Toledano Garrido']
EXH=['Marcos Calle Suárez','Juan Pedro Muñoz Miguel','Antonio Carmona Lavado','Manuel Villasalero Díaz','Javier Tamayo Torres','Juan Manuel Maqueira Marín']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5765=[ID,CD,EX,EXP]

ID=['DF5766','Profesoras y Profesores Titulares de Universidad','Organización de Emoresas','Ciencias Sociales y Jurídicas','Organización de Empresas','Facultad de Ciencias Económicas y Empresariales (Sede Algeciras)','Docencia en el área de conocimiento','Gestión del Capital Humano y Liderazgo en Equipos de Investigación']
CD=['','','']
EXM=['Beatriz Rodrigo Moya','Aurora Garrido Moreno','Marta Solorzano García','María del Sagrario Romero Cuadrado','Elisa Pilar Baraibar Díez','Rosalía Díaz Carrión']
EXH=['Pablo Ruiz Palomino','Roberto Luna Arocas','Manuel Villasalero Díaz','Julio Navío Marco','Eduardo Pérez Gorostegui','Álvaro López Cabrales']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5766=[ID,CD,EX,EXP]

ID=['DF5767','Profesoras y Profesores Titulares de Universidad','Organización de Emoresas','Ciencias Sociales y Jurídicas','Organización de Empresas','Facultad de Ciencias Económicas y Empresariales','Docencia en el área de conocimiento','Gestión del Capital Intelectual en Equipos Científicos']
CD=['','','']
EXM=['Mónica Santana Hernández','Raquel Sanz Valle','María Eugenia Sánchez Vidal','Susana Pasamar Reyes','Rosalía Díaz Carrión','María Pilar Jerez Gómez']
EXH=['Álvaro López Cabrales','Joaquín Alegre Vidal','Manuel Villasalero Díaz','Gregorio Sánchez Marín','José Luis Galán González','Luigi Stirpe']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5767=[ID,CD,EX,EXP]

ID=['DF5768','Profesoras y Profesores Titulares de Universidad','Psicobiología','Ciencias Sociales y Jurídicas','Psicología','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Investigación preclínica y traslacional en biomarcadores del neuroepitelio olfativo orientada al diagnóstico y/o tratamiento de trastornos afectivos']
CD=['','','']
EXM=['Laura Orio Ortiz','Olga Valverde Granados','Gemma Guillazo Blanch','Roser Nadal Alemany','Isabel María Martín Monzón','María Raquel Gómez de Heras']
EXH=['Emilio Ambrosio Flores','Pedro José Montoya Jiménez','Carlos María Gómez González','Manuel Portavella García','Manuel Vázquez Marrufo','Fernando Gámiz Ruiz']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5768=[ID,CD,EX,EXP]

ID=['DF5769','Profesoras y Profesores Titulares de Universidad','Psicología Social','Ciencias Sociales y Jurídicas','Psicología','Facultad de Ciencias Sociales y de la Comunicación','Docencia en el área de conocimiento','Aspectos psicosociales del consumidor de vino. Análisis actitudinal, biométrico, emocional, y experiencial (enoturístico).']
CD=['','','']
EXM=['Silvia Medina Anzano','Yolanda Troyano Rodríguez','Trinidad Núñez Domínguez','María Mercedes Durán Segura','María Isabel Mendoza Sierra','Yolanda Navarro Abal']
EXH=['José María León Pérez','José María León Rubio','Cristian Manuel Suárez Relinque','Roberto Martínez Pecino','Alejandro Orgambidez Ramos','Miguel Ángel Garrido Torres']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5769=[ID,CD,EX,EXP]

ID=['DF5770','Profesoras y Profesores Titulares de Universidad','Química Analítica','Ciencias','Química Analítica','Facultad de Ciencias','Docencia en el área de conocimiento','Química Analítica aplicada al análisis de elementos traza']
CD=['','','']
EXM=['María Gracia Bagur González','Ana María Ballesteros Gómez','Olga Busto Busto','Alegría Carrasco Pancorbo','María Ángeles Fernández Recamales','Elisa Isabel Vereda Alonso']
EXH=['Óscar Ballesteros García','Evaristo Antonio Ballesteros Tribaldo','Miguel Ángel Bello López','Antonio José Fernández Espinosa','Ricardo Javier López Gómez','Antonio Ruiz Medina']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5770=[ID,CD,EX,EXP]

ID=['DF5771','Profesoras y Profesores Titulares de Universidad','Química Analítica','Ciencias','Química Analítica','Facultad de Ciencias','Docencia en el área de conocimiento','Química Analítica aplicada al análisis de elementos traza']
CD=['','','']
EXM=['María Gracia Bagur González','Ana María Ballesteros Gómez','Olga Busto Busto','Alegría Carrasco Pancorbo','María Ángeles Fernández Recamales','Elisa Isabel Vereda Alonso']
EXH=['Óscar Ballesteros García','Evaristo Antonio Ballesteros Tribaldo','Miguel Ángel Bello López','Antonio José Fernández Espinosa','Ricardo Javier López Gómez','Antonio Ruiz Medina']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5771=[ID,CD,EX,EXP]

ID=['DF5772','Profesoras y Profesores Titulares de Universidad','Química Analítica','Ciencias','Química Analítica','Facultad de Ciencias','Docencia en el área de conocimiento','Desarrollo de metodología analítica rápida y su aplicación en Agroalimentación']
CD=['','','']
EXM=['Olga Busto Busto','María Purificación Fernández Zurbano','Carmen García Ruiz','María José Jara Palacios','Marta Lores Aguín','Merichel Plaza del Moral']
EXH=['Ricard Boqué Martí','José Elías Conde González','Luis Cuadros Rodríguez','José Luis Pérez Pavón','Esteban Alonso Álvarez','Féliz Zapata Arráez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5772=[ID,CD,EX,EXP]

ID=['DF5773','Profesoras y Profesores Titulares de Universidad','Química Orgánica','Ciencias','Química Orgánica','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Química Orgánica: Estudios de Productos Naturales con actividad alelopática']
CD=['','','']
EXM=['Ana María Estévez Braun','Isabel López Bazzocchi','María Isabel Montañez Vega','Yolanda Vida Pol','Patricia Ruiz Remón','Luisa Carlota López Cara']
EXH=['Ignacio Antonio Jiménez Díaz','Jose Javier Fernández Castro','Francisco Nájera Albendín','Ezequiel Pérez de Inestrosa Villatoro','Jesús Fernández Arteaga','José Antonio Gómez Vidal']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5773=[ID,CD,EX,EXP]

ID=['DF5774','Profesoras y Profesores Titulares de Universidad','Química Orgánica','Ciencias','Química Orgánica','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Química Orgánica: Estudios de Productos Naturales con actividad alelopática']
CD=['','','']
EXM=['Ana María Estévez Braun','Isabel López Bazzocchi','María Isabel Montañez Vega','Yolanda Vida Pol','Patricia Ruiz Remón','Luisa Carlota López Cara']
EXH=['Ignacio Antonio Jiménez Díaz','Jose Javier Fernández Castro','Francisco Nájera Albendín','Ezequiel Pérez de Inestrosa Villatoro','Jesús Fernández Arteaga','José Antonio Gómez Vidal']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5774=[ID,CD,EX,EXP]

ID=['DF5775','Profesoras y Profesores Titulares de Universidad','Química Orgánica','Ciencias','Química Orgánica','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Estudios de compuestos orgánicos con actividad sobre plantas parásitas']
CD=['','','']
EXM=['Ana María Estévez Braun','Isabel López Bazzocchi','María Isabel Montañez Vega','Yolanda Vida Pol','Patricia Ruiz Remón','Luisa Carlota López Cara']
EXH=['Ignacio Antonio Jiménez Díaz','Jose Javier Fernández Castro','Francisco Nájera Albendín','Jesús Fernández Arteaga','José Antonio Gómez Vidal','Francisco Franco Montalbán']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5775=[ID,CD,EX,EXP]

ID=['DF5776','Profesoras y Profesores Titulares de Universidad','Química Orgánica','Ciencias','Química Orgánica','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Aislamiento y determinación estructural de productos naturales marinos bioactivos']
CD=['','','']
EXM=['Miriam Álvarez Corral','Elena María Benito Hernández','Pilar García García','Yolanda Vida Pol','Patricia Ruiz Remón','María Araceli González Campaña']
EXH=['Ignacio López-Coca Martín','Antonio José Marchal Ingrain','José Francisco Quilez del Moral','Alberto Martínez Cuezva','Manuel Pernía Leal','Antonio José Ortiz Hernández']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5776=[ID,CD,EX,EXP]

ID=['DF5777','Profesoras y Profesores Titulares de Universidad','Sociología','Ciencias Sociales y Jurídicas','Economía General','Facultad de Ciencias del Trabajo','Docencia en el área de conocimiento','Trabajo, Desarrollo Humano, Desarrollo Sostenible']
CD=['','','']
EXM=['María Livia García Faroldi','Carmen Gómez Bueno','Estrella Gualda Caballero','María Luisa Jiménez Rodrigo','Nieves Ortega Pérez','Antonia María Ruiz Jiménez']
EXH=['Juan López Doblas','José Andrés Domínguez Gómez','José Manuel García Moreno','Rafael Martínez Martín','Rafael Merinero Rodríguez','Francisco Alberto Vallejo Peña']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5777=[ID,CD,EX,EXP]

ID=['DF5778','Profesoras y Profesores Titulares de Universidad','Trabajo Social y Servicios Sociales','Ciencias Sociales y Jurídicas','Derecho del Trabajo y de la Seguridad Social','Facultad de Ciencias del Trabajo','Docencia en el área de conocimiento','Violencia intrafamiliar y de género, inclusión y emprendimiento']
CD=['','','']
EXM=['Blanca Amalia Girela Rejón','Josefa Fombuena Valero','Aránzazu Gallego Molinero','Yolanda de los Ángeles Domenech López','María del Carmen Verde Diego','Ana Sales Ten']
EXH=['Jesús Muyor Rodríguez','Enrique Pastor Seller','Francesc Xavier Uceda Maza','Antonio López Peláez','Joan Lacomba Vázquez','Xaime Marcuello Servós']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5778=[ID,CD,EX,EXP]

ID=['DF5779','Profesoras y Profesores Titulares de Universidad','Derecho del Trabajo y de la Seguridad Social','Ciencias Sociales y Jurídicas','Derecho del Trabajo y de la Seguridad Social','Facultad de Ciencias del Trabajo','Docencia en el área de conocimiento','Investigación en Derecho del Trabajo y Derecho de la Seguridad Social']
CD=['','','']
EXM=['Carolina Martínez Moreno','Patrocinio Teresa Rodríguez Ramos','María Remedios Menéndez Calvo','Raquel Serrano Olivares','María José Gómez-Millán Herencia','María Luisa Pérez Guerrero']
EXH=['Eduardo López Ahumada','Xosé Manuel Carril Vázquez','Miguel Ángel Rodríguez-Piñero Royo','Francisco Andrés Valle Muñoz','Manuel Luque Parra','José Luis Lázaro Sánchez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5779=[ID,CD,EX,EXP]

ID=['DF5780','Profesoras y Profesores Titulares de Universidad','Derecho del Trabajo y de la Seguridad Social','Ciencias Sociales y Jurídicas','Derecho del Trabajo y de la Seguridad Social','Facultad de Ciencias del Trabajo','Docencia en el área de conocimiento','Investigación en Derecho del Trabajo y Derecho de la Seguridad Social']
CD=['','','']
EXM=['María José Romero Ródenas','Carolina Martínez Moreno','Carolina Gala Durán','Raquel Serrano Olivares','María Gema Quintero Lima','Esperanza Macarena Hernández Bejarano']
EXH=['Jaime Cabeza Pereiro','Eduardo López Ahumada','Francisco Javier Calvo Gallego','Francisco Lozano Lares','José Luis Lázaro Sánchez','Miguel Ángel Gómez Salado']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5780=[ID,CD,EX,EXP]

ID=['DF5781','Profesoras y Profesores Titulares de Universidad','Zoología','Ciencias','Biología','Facultad de Ciencias del Mar y Ambientales','Docencia en el área de conocimiento','Biología de la conservación y del movimiento en vertebrados voladores']
CD=['','','']
EXM=['Francisca del Carmen Ruano Díaz','Ana Carmen Durán Boyero','María Mar Torralva Forero','Carmen Hidalgo Jiménez','Zaida Ortega Diago','Marta Pola Pérez']
EXH=['José Manuel Tierno de Figueroa','Alfonso Marzal Reynolds','Jesús Domínguez Conde','Javier Balbontín Arenas','Alberto Luis Velando Rodríguez','Casimiro Corbacho Amado']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5781=[ID,CD,EX,EXP]

ID=['DF5782','Profesoras y Profesores Titulares de Universidad','Zoología','Ciencias','Biología','Facultad de Ciencias del Mar y Ambientales','Docencia en el área de conocimiento','Conservación de aves acuáticas y su función como biovectores']
CD=['','','']
EXM=['Amalia E. Morales Hernández','Ana Carmen Durán Boyero','Rosaura Durán Boyero','Amelia Victoria De Andrés Fernández','Mercedes Conradi Barrena','Yolanda Fernández Torquemata']
EXH=['José Ramón Arrebola Burgos','Juan Carlos Pérez Quintero','Jose Manuel Guerra García','Miguel Gaju Ricart','César Megina Martínez','Javier Calzada Samperio']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5782=[ID,CD,EX,EXP]

ID=['DF5789','Profesoras y Profesores Titulares de Universidad. Categoría asistencial: Facultativo Especialista de área','Medicina','Ciencias de la Salud','Medicina y Cirugía','Facultad de Medicina. Centro asistencial: Hospital Universitario de Jerez de la Frontera','Docencia en el área de conocimiento. Especialidad/perfil asistencial: Medicina Intensiva','Decisiones éticas e infecciones severas en el paciente crítico']
CD=['','','']
EXM=['María Concepción Carratala Munuera', 'María de la Luz Cuadrado Pérez','Eva María González Barca','Rocío González-Conejero Hilla','María Luisa Pedro-Botet Montoya','Ana Isabel Sánchez Fructuoso']
EXH=['Amado Andrés Belmonte','Francisco Javier Delgado Lista','Ricardo Gómez Huelgas','Miguel León Sanz','Juan Pedro-Botet Montoya','José Antonio Serra Rexach']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5789=[ID,CD,EX,EXP]

ID=['DF5790','Profesoras y Profesores Titulares de Universidad. Categoría asistencial: Facultativo Especialista de área','Pediatría','Ciencias de la Salud','Materno-Infantil y Radiología','Facultad de Medicina. Centro asistencial: Hospital Universitario Puerta del Mar','Docencia en el área de conocimiento. Especialidad/perfil asistencial: Pediatría','Neurología Neonatal']
CD=['','','']
EXM=['María Gloria Bueno Lozano','Cristina Campoy Folgoso','Julia Colomer Revuelta','María de la Luz Couce Pico','María Mercedes Gil Campos','Gloria González Azpeitia']
EXH=['Ignacio Jesús Dávila González','José Ramón Castro Conde','José Manuel Marugán de Miguelsanz','Carlos Jesús Ruiz Cosano', 'Manuel Sánchez Luna','José Uberos Fernández']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5790=[ID,CD,EX,EXP]

DFTU=[DF5223,DF5723,DF5724,DF5725,DF5726,DF5727,DF5728,DF5729,DF5730,DF5731,DF5732,DF5733,DF5734,DF5735,DF5736,DF5737,DF5738,DF5739,DF5740,DF5741,DF5742,DF5743,DF5745,DF5746,DF5747,DF5748,DF5749,DF5750,DF5751,DF5752,DF5753,DF5754,DF5755,DF5756,DF5757,DF5758,DF5759,DF5760,DF5761,DF5762,DF5763,DF5764,DF5765,DF5766,DF5767,DF5768,DF5769,DF5770,DF5771,DF5772,DF5773,DF5774,DF5775,DF5776,DF5777,DF5778,DF5779,DF5780,DF5781,DF5782,DF5789,DF5790]

ID=['DF5677','Catedráticas y Catedráticos de Universidad','Filología Latina','Arte y Humanidades','Filología Clásica','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Humanismo latino hispano-italiano. Epistolografía y poesía neolatinas. Plaza a convocar por cupo general para personas con discapacidad']
CD=['','','']
EXM=['Trinidad Arcos Pereira','Estrella del Carmen Pérez Rodríguez','María Luisa Harto Trujillo','Francisca del Mar Plaza Picón','Ana Isabel Martín Ferreira','María Isabel Velázquez Soriano']
EXH=['Luis María Merino Jerez','Francisco Salas Salgado','Manuel Mañas Núñez','Jorge Fernández López','Juan Francisco Mesa Sanz','Gregorio Rodríguez Herrera']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5677=[ID,CD,EX,EXP]

ID=['DF5678','Catedráticas y Catedráticos de Universidad','Geodinámica Externa','Ciencias','Ciencias de la Tierra','Facultad de Ciencias del Mar y Ambientales','Docencia en el área de conocimiento','Geomorfología litoral y riesgos naturales costeros']
CD=['','','']
EXM=['María Luisa Calvache Quesada','Montserrat Jiménez Sánchez','María José Domínguez Cuesta','Ana María Bernabeu Tello','Rosa María Mateos Ruiz','Gloria Cuenca Bescós']
EXH=['Javier Lario Gómez','Ignacio Antigüedad Auzmendi','Bartolomé Andreo Navarro','José Francisco Martín Duque','José Miguel Azañón Hernández','Juan Antonio Morales González']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5678=[ID,CD,EX,EXP]

ID=['DF5679','Catedráticas y Catedráticos de Universidad','Sociología','Ciencias Sociales y Jurídicas','Economía General','Facultad de Ciencias del Trabajo','Docencia en el área de conocimiento','Sociología del Trabajo y del Empleo']
CD=['','','']
EXM=['Estrella Gualda Caballero','María Dolores Cáceres Zapatero','Elisabet Almeda Samaranch','Almudena Moreno Mínguez','María Luz Morán Calvo-Sotelo','María Isabel Jociles Rubio']
EXH=['Enrique Martín Criado','Manuel Javier Callejo Gallego','Juan Sebastián Fernández Prados','Clemente Jesús Navarro Yáñez','Antonio Trinidad Requena','Gonzalo Vicente Herranz de Rafael']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5679=[ID,CD,EX,EXP]

ID=['DF5684','Catedráticas y Catedráticos de Universidad','Lenguajes y Sistemas Informáticos','Ingeniería y Arquitectura','Ingeniería Informática','Escuela Superior de Ingeniería','Docencia en el área de conocimiento','Tecnologías de Simulación y Gamificación en la Ingeniería del Sortware']
CD=['','','']
EXM=['Coral Calero Muñoz','María Teresa Gómez López','Coromoto Antonia León Hernández','Elena María Navarro Martínez','Alicia Troncoso Lora','Alejandra Martínez Monés']
EXH=['José Javier Dolado Cosín','Félix Óscar García Rubio','Óscar Pastor López','José Cristóbal Riquelme Santos','César Emilio Insfrán Pelozo','José Ambrosio Toval Álvarez']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5684=[ID,CD,EX,EXP]

ID=['DF5784','Catedráticas y Catedráticos de Universidad','Psicología Social','Ciencias Sociales y Jurídicas','Psicología','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Inteligencia Emocional y factores de adaptación psicosocial y calidad de vida en estudiantes y pacientes con cáncer de mama']
CD=['','','']
EXM=['Sofía López Roig','María Ángeles Pastor Mira','María Luisa Salanova Soria','María Isabel Hombrados Mendieta','Carmen Pozo Muñoz','Victoria Aurora Ferrer Pérez']
EXH=['Ángel Barrasa Notrario','Luis Gómez Jacinto','Darío Nuño Díaz Méndez','Pablo Antonio Briñol Turnes','José Manuel Sabucedo Cameselle','José Carlos Sánchez García']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5784=[ID,CD,EX,EXP]

ID=['DF5785','Catedráticas y Catedráticos de Universidad','Psicología Básica','Ciencias Sociales y Jurídicas','Psicología','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Aspectos neurocognitivos y psicológicos en Psicología Aeroespacial en ambientes análogos espaciales y adaptación a ambiente extremo']
CD=['','','']
EXM=['Nuria Sebastián Gallés','Carmen Torres Bares','María Isabel de Brugada Sauras','María del Carmen Moreno Rodríguez','Inmaculada Montoya Castilla','Pilar Flores Cubos']
EXH=['Pablo Fernández Berrocal','Natalio Extremera Pacheco','Ricardo Pellón Suárez de Puga','José María León Rubio','Gonzalo de la Casa Rivas','Fernando Maestú Unturbe']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5785=[ID,CD,EX,EXP]

ID=['DF5786','Catedráticas y Catedráticos de Universidad','Lingüística General','Arte y Humanidades','Filología','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Lingüística Computacional']
CD=['','','']
EXM=['María Carmen Fernández Juncal','Milagros Fernández Pérez','Carmen Galán Rodríguez','Beatriz Gallardo Paúls','Sonia Madrid Cánovas','Luisa Martín Rojo']
EXH=['Francisco de Asís Fernández García','Carlos Hernández Sacristán','Ricardo Morant Marco','Francisco José Salguero Lamillar','Enrique Nicolás Serra Alegre','Alfonso Zamorano Aguilar']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5786=[ID,CD,EX,EXP]

ID=['DF5787','Catedráticas y Catedráticos de Universidad','Bioquímica y Biología Molecular','Ciencias','Biomedicina, Biotecnología y Salud Pública','Facultad de Medicina','Docencia en el área de conocimiento','Proteómica Traslacional en Biomedicina']
CD=['','','']
EXM=['Rocío Martínez de Pablos','Carmen Michán Doña','María Dolores Girón González','Laura Herrero Rodríguez','Marta Giralt Oms','Teresa Portolés Pérez']
EXH=['Miguel Ángel Medina Torres','Luis Fontana Gallego','Rafael Salto González','Isaac Túnez Fiñana','José Luis Venero Recio','Manuel José Rodríguez Ortega']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5787=[ID,CD,EX,EXP]

ID=['DF5788','Catedráticas y Catedráticos de Universidad. Categoría asistencial: Facultativo Especialista de área','Pediatría','Ciencias de la Salud','Materno-Infantil y Radiología','Facultad de Medicina. Centro asistencial: Hospital Universitario Puerta del Mar','Docencia en el área de conocimiento. Especialidad/perfil asistencial: Pediatría','Endocrinología Pediátrica']
CD=['','','']
EXM=['Gloria Bueno Lozano','Mercedes Gil Campos','María Rosaura Leis Trabazo','Pilar Codoñer Franch','María de la Luz Couce Pico','Cecilia Martínez Costa']
EXH=['Jesús Argente Oliver','Luis Antonio Castaño González','Corsino Rey Galán','Enrique Galán Gómez','José Tomás Ramos Amador','Feliciano Jesús Ramos Fuentes']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5788=[ID,CD,EX,EXP]

ID=['DF5798','Catedráticas y Catedráticos de Universidad','Filología Francesa','Arte y Humanidades','Filología Francesa e Inglesa','Facultad de Filosofía y Letras','Docencia en el área de conocimiento','Literatura y Cultura del África Francófona']
CD=['','','']
EXM=['María Lourdes Carriedo López','María Mercedes Eurrutia','Amelia Gamoneda Lanza','María Isabel Peñalver Vicea','Lourdes Ángeles Terrón Barbosa','Lidia Vázquez Jiménez']
EXH=['Antonio Ansón Anadón','Antonio Bueno García','Francisco González Fernández','Ignacio Iñarrea Heras','Evelio Miñano Martínez','José Ignacio Ramos Gay']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5798=[ID,CD,EX,EXP]

ID=['DF5799','Catedráticas y Catedráticos de Universidad','Psicología Evolutiva y de la Educación','Ciencias Sociales y Jurídicas','Psicología','Facultad de Ciencias de la Educación','Docencia en el área de conocimiento','Consecuencias psicológicas durante el ciclo vital de las conductas adictivas y/o la violencia de género']
CD=['','','']
EXM=['María del Carmen Pichardo Martínez','María del Carmen Moreno Rodríguez','Beatriz Delgado Doménech','María Victoria Hidalgo García','Rosario Isabel Ferrer Cascales','María Teresa Daza González']
EXH=['Juan Luis Luque Vilaseca','Javier Rosales Pardo','Santiago Pelegrina López','Eduardo Fonseca Pedrero','Francisco Cruz Quintana','José Javier López Goñi']
EX=[EXM,EXH]
EXP=[committee(EXM,EXH,6)[0],committee(EXM,EXH,6)[1]]
DF5799=[ID,CD,EX,EXP]

DFCU=[DF5677,DF5678,DF5679,DF5684,DF5784,DF5785,DF5786,DF5787,DF5788,DF5798,DF5799]

DF=DFCU+DFTU
DFS=extract(extract(DF))

D=[DFCU,DFTU,DCPPL,DCPAD]
CAT=['DC: CU','DF: TU','DC: PPL','DC: PAD']
TP=['TITULAR','SUPLENTE 1','SUPLENTE 2']
TPC=['Código','Área de Conocimiento','TITULAR','SUPLENTE 1','SUPLENTE 2']
GN=['PROFESORAS','PROFESORES']

#PATH="/Users/pacheco/downloads/"
#display(Image(filename=PATH+"uca.png",width=300,height=100))
display(Image(url='https://gabcomunicacion.uca.es/wp-content/uploads/2017/05/Logo-V1-Color-Web-600x237-pix-gif.gif?u',width=300,height=100,unconfined=True))
#display(Image(url='https://gabcomunicacion.uca.es/wp-content/uploads/2024/10/logo-color-negativo.svg?u',width=300,height=100,unconfined=True))

@interact
def raffle(code=widgets.Dropdown(options=['']+DFS+DCS,value='',description='Información',disabled=False),category=widgets.Dropdown(options=['']+CAT,value='',description='Categoría',disabled=False),raffle=widgets.Checkbox(value=False,description='Sortear',disabled=False,indent=False)):
    for i in range(len(DF)):
        if code==DFS[i]:
            print('Código:',DF[i][0][0])
            print('Cuerpo:',DF[i][0][1])
            print('Área de conocimiento:',DF[i][0][2])
            print('Rama de conocimiento:',DF[i][0][3])
            print('Departamento:',DF[i][0][4])
            print('Centro:',DF[i][0][5])
            print('Actividades a realizar:',DF[i][0][6])
            print('Línea de investigación preferente:',DF[i][0][7])
            tablita(DF[i][2],cons('',6),['Profesoras','Profesores'])
    if code=='':
        print()
    for i in range(len(DC)):
        if code==DCS[i]:
            print('Código:',DC[i][0][0])
            print('Categoría:',DC[i][0][1])
            print('Área de conocimiento:',DC[i][0][2])
            print('Departamento:',DC[i][0][3])
            print('Centro:',DC[i][0][4])
            print('Perfil Docente:',DC[i][0][5])
            print('Perfil Investigador:',DC[i][0][6]) 
            print('Puntuación mímina:',DC[i][0][7])
            tablita(DC[i][2],cons('',6),['Profesoras','Profesores'])
    if code=='':
        print()
    for i in range(len(D)):
        if category==CAT[i]:
            R=compco(D[i],3,3)
            G=[]
            for j in range(len(D[i])):
                G=G+[[D[i][j][0][0],D[i][j][0][2]]+R[j][0],['','']+R[j][1]]
            if raffle==True:
                tablita(G,TPC,cons('',len(G))).to_csv('raffle result.csv')
    if category=='':
        print()

interactive(children=(Dropdown(description='Información', options=('', 'DF5677', 'DF5678', 'DF5679', 'DF5684',…